In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('../../../data/miner/09517617000153_3.csv', date_parser = 'data')

In [3]:
data.head()

,data,produtoId,descr,valor,quantidade,valorCancelado,quantidadeCancelada,quantidadeDocumentos,quantidadeValorZero
0,2019-01-02,1001,CREME DENTAL CLOSEUP TRIPLE 70G MENTA,2.79,1.0,0.00,0.0,1,0.0
1,2019-01-02,10012802,MASTRUZ HORTA VERDE UN,19.90,10.0,3.98,2.0,9,0.0
2,2019-01-02,10022,LIXA UNHA M BONI 6 UND,2.59,1.0,0.00,0.0,1,0.0
3,2019-01-02,100328,POLVILHO YOKI 500G AZEDO,9.49,1.0,0.00,0.0,1,0.0
4,2019-01-02,100335,SHAMPOO PROCAO 500ML PELOS CLAROS,7.99,1.0,0.00,0.0,1,0.0


In [4]:
data['descr'] 

0             CREME DENTAL CLOSEUP TRIPLE 70G MENTA
1                            MASTRUZ HORTA VERDE UN
2                            LIXA UNHA M BONI 6 UND
3                          POLVILHO YOKI 500G AZEDO
4                 SHAMPOO PROCAO 500ML PELOS CLAROS
                             ...                   
1029898                        REFRIGERANTE KUAT 1L
1029899           REFRIGERANTE SUKITA 350ML LARANJA
1029900    ESCOVA DENTAL ORAL-B PRO.S 7B 40 MC L2P1
1029901         REFRI GUARANA ANTARCTICA 350ML LATA
1029902                          CERVEJA SKOL 350ML
Name: descr, Length: 1029903, dtype: object

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1029903 entries, 0 to 1029902
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   data                  1029903 non-null  object 
 1   produtoId             1029903 non-null  int64  
 2   descr                 1029446 non-null  object 
 3   valor                 1029903 non-null  float64
 4   quantidade            1029903 non-null  float64
 5   valorCancelado        1029903 non-null  float64
 6   quantidadeCancelada   1029903 non-null  float64
 7   quantidadeDocumentos  1029903 non-null  int64  
 8   quantidadeValorZero   1029903 non-null  float64
dtypes: float64(5), int64(2), object(2)
memory usage: 70.7+ MB


In [6]:
data['data'] = data['data'].astype('datetime64[ns]')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1029903 entries, 0 to 1029902
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   data                  1029903 non-null  datetime64[ns]
 1   produtoId             1029903 non-null  int64         
 2   descr                 1029446 non-null  object        
 3   valor                 1029903 non-null  float64       
 4   quantidade            1029903 non-null  float64       
 5   valorCancelado        1029903 non-null  float64       
 6   quantidadeCancelada   1029903 non-null  float64       
 7   quantidadeDocumentos  1029903 non-null  int64         
 8   quantidadeValorZero   1029903 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(2), object(1)
memory usage: 70.7+ MB


In [7]:
data.index = data['data']

In [8]:
CARNAVAL      = data.loc['2019-02-28':'2019-03-07']
SEMANA_SANTA  = data.loc['2019-04-14':'2019-04-20']
NATAL         = data.loc['2019-12-22':'2019-12-27']
ANO_NOVO      = data.loc['2019-12-29':'2020-01-03']

In [9]:
data['descr'].groupby(data['quantidade']).value_counts()

quantidade  descr                                          
0.0         CEREAL BARRA SUPINO ZERO CHOCOLATE AO LEITE 24G    199
            NECTAR SUFRESH 200ML MACA                           87
            KIT DESODORANTE AER REXONA MEN ACTIVE LV+PG-        77
            MASCARA CAPILAR GOLD 430G MEGA BRIL                 61
            PAO COCO MAX PAES 500G                              55
                                                              ... 
3936.0      CERVEJA BUDWEISER 350ML LATA                         1
4117.0      CERVEJA BOHEMIA 269ML LT                             1
5085.0      CERVEJA BUDWEISER 350ML LATA                         1
5360.0      CERVEJA BUDWEISER 350ML LATA                         1
6844.0      CERVEJA SKOL 350ML PURO MALTE                        1
Name: descr, Length: 156407, dtype: int64

In [10]:
def is_var_holliday(id_prod):#id_ass, id_loja)
    
    data_prod         = data[data['produtoId'] == id_prod]
    data_prod['data'] = data_prod['data'].astype('datetime64[ns]')
    
    
    CARNAVAL      = data_prod.loc['2019-02-27':'2019-03-06']
    SEMANA_SANTA  = data_prod.loc['2019-04-13':'2019-04-20']
    NATAL         = data_prod.loc['2019-12-23':'2019-12-25']
    ANO_NOVO      = data_prod.loc['2019-12-30':'2020-01-01']
    
    dict_hollidays = {'Carnaval'    : CARNAVAL,
                      'Semana Santa': SEMANA_SANTA,
                      'Natal'       : NATAL,
                      'Ano Novo'    : ANO_NOVO}
    
    response = {}
    
    for key, value in dict_hollidays.items():
        response[key] = sense_product(data_prod, value)
    
    return response

In [11]:
def sense_product(data, data_holliday):
    
    data = pd.concat([data, data_holliday]).drop_duplicates(keep=False)
    
    baseline_data          = data['quantidade'].mean() + data['quantidade'].std()
    baseline_data_holliday = data_holliday['quantidade'].mean() + data_holliday['quantidade'].std()
    
    if (baseline_data < baseline_data_holliday):
        return True
    
    return False

In [12]:
# POLPA ELEGE 540G MORANGO id = 127929
is_var_holliday(127929)

{'Carnaval': False, 'Semana Santa': False, 'Natal': False, 'Ano Novo': True}